In [2]:
import pandas as pd
import os
import geopandas as gpd

In [2]:
folder_path = "E:\D-Drive\Dynamic_world_1\data_India_all_2021"

In [3]:
summary_data = []

In [4]:
# Process each file
for filename in os.listdir(folder_path):
    if filename.endswith(".csv"):
        filepath = os.path.join(folder_path, filename)
        df = pd.read_csv(filepath)

        # Get HYBAS_ID and total_area (same for all rows)
        hybas_id = df["HYBAS_ID"].iloc[0]
        total_area = df["total_area"].iloc[0]

        # Separate monthly data (exclude '2021_total')
        monthly_df = df[~df["Period"].str.contains("total")]

        # Compute stats on monthly coverage percent
        monthly_mean = monthly_df["Coverage_percent"].mean()
        monthly_min = monthly_df["Coverage_percent"].min()
        monthly_max = monthly_df["Coverage_percent"].max()
        monthly_variation = monthly_max - monthly_min

        # Get yearly value (row where Period = 2021_total)
        yearly_row = df[df["Period"] == "2021_total"]
        yearly_value = yearly_row["Coverage_percent"].values[0] if not yearly_row.empty else None

        # Append summary
        summary_data.append({
            "HYBAS_ID": hybas_id,
            "monthly_mean": monthly_mean,
            "monthly_min": monthly_min,
            "monthly_max": monthly_max,
            "monthly_variation": monthly_variation,
            "yearly_value": yearly_value,
            "total_area": total_area
        })

# Convert to DataFrame
summary_df = pd.DataFrame(summary_data)


In [5]:
summary_df.head(1)

,HYBAS_ID,monthly_mean,monthly_min,monthly_max,monthly_variation,yearly_value,total_area
0,4050024190,65.999725,2.249873,100.0,97.750127,100.0,1.039200e+07


In [6]:
summary_df.to_csv("Indian_basins_summary_2021.csv", index=False)

In [ ]:
!pip install geopandas matplotlib

In [7]:
# Path to your shapefile folder
hydrobasin_path = "E:\D-Drive\Dynamic_world_1\hybas_as_lev05_v1c"  

# Read the shapefile (usually the .shp file is inside the folder)
gdf_basins = gpd.read_file(os.path.join(hydrobasin_path, "hybas_as_lev05_v1c.shp"))  

# View the columns (to find the HYBAS_ID field name, usually HYBAS_ID)
print(gdf_basins.columns)

Index(['HYBAS_ID', 'NEXT_DOWN', 'NEXT_SINK', 'MAIN_BAS', 'DIST_SINK',
       'DIST_MAIN', 'SUB_AREA', 'UP_AREA', 'PFAF_ID', 'ENDO', 'COAST', 'ORDER',
       'SORT', 'geometry'],
      dtype='object')


In [9]:
gdf_basins["HYBAS_ID"] = gdf_basins["HYBAS_ID"].astype(int)
summary_df["HYBAS_ID"] = summary_df["HYBAS_ID"].astype(int)

# Merge GeoDataFrame with summary stats
gdf_merged = gdf_basins.merge(summary_df, on="HYBAS_ID", how="inner")

In [10]:
gdf_merged.head(1)

,HYBAS_ID,NEXT_DOWN,NEXT_SINK,MAIN_BAS,DIST_SINK,DIST_MAIN,SUB_AREA,UP_AREA,PFAF_ID,ENDO,COAST,ORDER,SORT,geometry,monthly_mean,monthly_min,monthly_max,monthly_variation,yearly_value,total_area
0,4051019890,4050023810,4050023810,4050023810,760.1,760.1,114686.9,114686.9,44582,0,0,2,328,"POLYGON ((95.29167 21.55, 95.29231 21.51036, 9...",62.072792,0.234871,99.881567,99.646696,100.0,1.264386e+07


In [11]:
# Export merged GeoDataFrame to GeoJSON for geemap
geojson_path = "merged_hydrobasins_India.geojson"
gdf_merged.to_file(geojson_path, driver="GeoJSON")

In [3]:

# Load your GeoDataFrame
gdf = gpd.read_file("merged_hydrobasins_India.geojson")

# Confirm column names
gdf.columns

Index(['HYBAS_ID', 'NEXT_DOWN', 'NEXT_SINK', 'MAIN_BAS', 'DIST_SINK',
       'DIST_MAIN', 'SUB_AREA', 'UP_AREA', 'PFAF_ID', 'ENDO', 'COAST', 'ORDER',
       'SORT', 'monthly_mean', 'monthly_min', 'monthly_max',
       'monthly_variation', 'yearly_value', 'total_area', 'geometry'],
      dtype='object')

In [4]:
import geopandas as gpd
import geemap
import ipywidgets as widgets
from ipyleaflet import GeoJSON, WidgetControl, Popup
from branca.colormap import linear
import matplotlib.pyplot as plt
from matplotlib import cm, colors 

def plot_choropleth(feature, cmap_name='viridis'):
    # Step 1: Clean data
    gdf_plot = gdf[["HYBAS_ID", "geometry", feature]].dropna()

    # Step 2: Colormap setup
    vmin, vmax = gdf_plot[feature].min(), gdf_plot[feature].max()
    colormap = linear.__getattribute__(cmap_name).scale(vmin, vmax)
    colormap.caption = feature.replace("_", " ").title()
    norm = plt.Normalize(vmin=vmin, vmax=vmax)
    cmap = cm.get_cmap(cmap_name)
    
    # Step 3: Convert GeoDataFrame to styled GeoJSON
    styled_features = []
    id_lookup = {}
    for _, row in gdf_plot.iterrows():
        value = round(row[feature], 2)
        color = colormap(value)
        hybas_id = int(row["HYBAS_ID"])
        props = {
            "HYBAS_ID": hybas_id,
            feature: value,
            "style": {
                "color": "black",
                "weight": 0.5,
                "fillColor": color,
                "fillOpacity": 0.8
            }
        }
        id_lookup[hybas_id] = props  # Store for popup access
        styled_features.append({
            "type": "Feature",
            "geometry": row["geometry"].__geo_interface__,
            "properties": props
        })

    geojson_data = {
        "type": "FeatureCollection",
        "features": styled_features
    }

    gdf_plot["style"] = gdf_plot[feature].apply(lambda x: colors.to_hex(cmap(norm(x))))

    features_json = []
    for _, row in gdf_plot.iterrows():
        features_json.append({
            "type": "Feature",
            "geometry": row["geometry"].__geo_interface__,
            "properties": {
                "HYBAS_ID": row["HYBAS_ID"],
                feature: row[feature],
                "style": {
                    "color": "black",
                    "weight": 0.5,
                    "fillColor": row["style"],
                    "fillOpacity": 0.8
                }
            }
        })

    geojson_dict = {
        "type": "FeatureCollection",
        "features": features_json
    }
    # Step 4: Create the map
    m = geemap.Map(center=[22.7266, 74.9799], zoom=4)

    # Step 5: Add the GeoJSON layer
    geo_json_layer = GeoJSON(
        data=geojson_data,
        style={"opacity": 1, "fillOpacity": 0.7},
        hover_style={"fillColor": "white", "fillOpacity": 0.4},
        name=feature
    )
    m.add_layer(geo_json_layer)
    m.add_geojson(geojson_dict, layer_name=feature, info_mode="on_hover")

    # Step 7: Add color legend
    legend_html = colormap._repr_html_()
    legend_widget = widgets.HTML(value=legend_html)
    legend_control = WidgetControl(widget=legend_widget, position='bottomright')
    m.add_control(legend_control)


    return m


In [5]:
plot_choropleth("monthly_mean")

C:\Users\Satbir\AppData\Local\Temp\ipykernel_22760\958187023.py:18: MatplotlibDeprecationWarning: The get_cmap function was deprecated in Matplotlib 3.7 and will be removed in 3.11. Use ``matplotlib.colormaps[name]`` or ``matplotlib.colormaps.get_cmap()`` or ``pyplot.get_cmap()`` instead.
  cmap = cm.get_cmap(cmap_name)


Map(center=[22.7266, 74.9799], controls=(WidgetControl(options=['position', 'transparent_bg'], widget=SearchDa…

In [6]:
features = ["monthly_mean", "monthly_min", "monthly_max", "monthly_variation", "yearly_value"]

for feature in features:
    print(f"Interactive map for {feature}")
    display(plot_choropleth(feature))

Interactive map for monthly_mean


Map(center=[22.7266, 74.9799], controls=(WidgetControl(options=['position', 'transparent_bg'], widget=SearchDa…

Interactive map for monthly_min


Map(center=[22.7266, 74.9799], controls=(WidgetControl(options=['position', 'transparent_bg'], widget=SearchDa…

Interactive map for monthly_max


Map(center=[22.7266, 74.9799], controls=(WidgetControl(options=['position', 'transparent_bg'], widget=SearchDa…

Interactive map for monthly_variation


Map(center=[22.7266, 74.9799], controls=(WidgetControl(options=['position', 'transparent_bg'], widget=SearchDa…

Interactive map for yearly_value


Map(center=[22.7266, 74.9799], controls=(WidgetControl(options=['position', 'transparent_bg'], widget=SearchDa…